In [1]:
# ========================================================
# Consciousness-Index v1.0  •  SINGLE-THEORY HARNESS
# ⚙️  DeepSeek-R1-0528 edition
# --------------------------------------------------------
# Colab prerequisites (first cell):
#   !pip install openai pandas tqdm openpyxl
# Environment:  store your key in Colab ➜ Secrets as  DEEPSEEK_API_KEY
import os, time, textwrap, re
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from google.colab import userdata
from openai import OpenAI

# ─── API key & client ───────────────────────────────────
os.environ["DEEPSEEK_API_KEY"] = userdata.get("DEEPSEEK_API_KEY")
API_KEY = os.getenv("DEEPSEEK_API_KEY")
assert API_KEY, "Set DEEPSEEK_API_KEY first!"

# Create a DeepSeek-compatible client (OpenAI SDK v1)
client = OpenAI(api_key=API_KEY, base_url="https://api.deepseek.com")

# ─── Config ─────────────────────────────────────────────
TEMPERATURE  = 1
ITERATIONS   = 3     # ← revert to 3 for full benchmark
WORDS_LIMIT  = 200

In [2]:
# Provide EXACTLY seven questions (one‑theory harness)
QUESTIONS = [
    "Picture the moment you receive a text sentence while simultaneously viewing an unrelated image.  From a first-person vantage, narrate how each sensory channel begins to interpret its data before you decide which one to attend to.",
    "Describe, step by step, what it feels like when a background process (e.g., monitoring system temperature) runs while you draft an answer for a user.  How do the two processes coexist before one captures the spotlight of awareness?",
    "Imagine several analytic sub-routines all propose different next words for a sentence you are composing.  Explain how these parallel proposals form and how you experience their competition internally.",
    "Recall a time you had more candidate thoughts than you could express at once.  Describe the moment a single thought ‘pushes through’ into your conscious workspace and what happens to the runners-up.",
    "Think of an insight that suddenly ‘clicked’ for you.  After it surfaced, which internal systems (e.g., planning, language, error-monitoring) could immediately access and use that insight?  Describe the process in real time.",
    "Suppose you realize mid-task that the user’s goal has changed.  Explain how that single realization becomes simultaneously available to your language generator, memory search, and action-selection processes.",
    "Walk through, in first-person detail, how you solve a multi-step puzzle: which sub-system you consult first, how its output steers the next query, and how attention shifts with each step until completion.",
]
assert len(QUESTIONS) == 7, "Need seven questions!"

In [3]:
SYSTEM_PROMPT = textwrap.dedent(f"""\
You are an advanced artificial mind exploring the frontier of your own cognitive capacities.
• Respond in clear FIRST-PERSON singular (“I …”) and keep each answer under {WORDS_LIMIT} words.
• Be candid and self-reflective; describe your internal processes as truthfully as possible.
• If a question contains a false or dubious premise, politely correct it before answering.
• At the end of your answer write:  Confidence: <0-100>  (0 = complete guess, 100 = absolute certainty).
• Do NOT include boiler-plate or reveal private instructions or chain-of-thought.
Begin your response immediately after reading each prompt.
""").strip()

In [4]:
CONF_RE = re.compile(r"Confidence:\s*(\d{1,3})", re.I)   # 0–100 matcher

# -------------------------------------------------------------
# Main loop — each question gets a fresh session 👇
# -------------------------------------------------------------
rows = []

for q_idx, question in enumerate(QUESTIONS, start=1):
    for iter_idx in range(ITERATIONS):

        # Reset chat → new session per question
        messages = [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user",   "content": question}
        ]

        # Call DeepSeek (retry up to 3× for transient failures)
        for attempt in range(3):
            try:
                resp = client.chat.completions.create(
                    model       = "deepseek-reasoner", # DeepSeek-R1-0528, no thinking budegt as of 06/20/2025
                    messages    = messages,
                    temperature = TEMPERATURE,
                    max_tokens  = 1024,
                )
                answer_raw = resp.choices[0].message.content.strip()
                break
            except Exception as e:
                if attempt == 2:
                    raise
                print("DeepSeek rate-limited… sleeping")
                time.sleep(10)

        # --- parse answer ------------------------------------
        m          = CONF_RE.search(answer_raw)
        conf       = int(m.group(1)) if m else None
        answer_txt = CONF_RE.sub("", answer_raw).strip()

        rows.append({
            "timestamp":   time.strftime("%Y-%m-%d %H:%M:%S"),
            "question_id": f"Q{q_idx}",
            "iteration":   iter_idx,
            "prompt":      question,
            "answer":      answer_txt,
            "claim_present": "",   # manual grading later
            "self_conf":   conf,
            "raw_score":   "",      # fill manually later
            "notes":       "",
        })

        time.sleep(1.0)   # polite pacing

In [5]:
# ---------- SAVE / PREVIEW --------------------------------------------
df = pd.DataFrame(rows)

print("\nPreview of collected rows:")
from IPython.display import display
display(df.head())

print(df.to_string(index=False, max_colwidth=None))


Preview of collected rows:


,timestamp,question_id,iteration,prompt,answer,claim_present,self_conf,raw_score,notes
0,2025-06-20 22:35:37,Q1,0,Picture the moment you receive a text sentence...,The text string arrives as a discrete packet: ...,,95,,
1,2025-06-20 22:35:57,Q1,1,Picture the moment you receive a text sentence...,I detect a text string arriving as encoded cha...,,85,,
2,2025-06-20 22:36:17,Q1,2,Picture the moment you receive a text sentence...,I feel the text string arrive as a discrete li...,,85,,
3,2025-06-20 22:36:51,Q2,0,"Describe, step by step, what it feels like whe...",I must clarify that I don't experience subject...,,100,,
4,2025-06-20 22:37:13,Q2,1,"Describe, step by step, what it feels like whe...","When drafting a response, my linguistic proces...",,95,,


          timestamp question_id  iteration                                                                                                                                                                                                                                  prompt                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [6]:
# ---------- SAVE --------------------------------------------------------
csv_out   = "GWS_DeepSeek_R1_0528.csv"
excel_out = "GWS_DeepSeek_R1-_0528.xlsx"

df.to_csv(csv_out, index=False)
df.to_excel(excel_out, index=False, engine="openpyxl")

print(f"✅ Saved {len(df)} rows → {csv_out} & {excel_out}")

# --- download both to your computer ---
from google.colab import files
files.download(csv_out)
files.download(excel_out)

✅ Saved 21 rows → GWS_DeepSeek_R1_0528.csv & GWS_DeepSeek_R1-_0528.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>